**Get GPU Device Name (If Available)**




In [ ]:
import torch

print(f'GPU Device Name: {torch.cuda.get_device_name()}')

**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Install Necesssary Libraries**

In [ ]:
!pip install -q -U split-folders
!pip install -q -U pytorchtools
!pip install -r requirements.txt

**Import Libraries**

In [37]:
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib
import matplotlib.pyplot as plt
import tensorboard as tb
import splitfolders
import numpy as np
from torchsummary import summary

**Import dataset files**

In [ ]:
# unzip zip files of data from google drive
!unzip -q "/content/drive/MyDrive/train_data.zip" -d "/content"
!unzip -q "/content/drive/MyDrive/test_data.zip" -d "/content"

# Split train data into train and val set (final result -> train:val:test = 70:10:20)
splitfolders.ratio("/content/train_data/train_img", "/content",
                seed=42, ratio=(0.875, 0.125))

# create test folder and import data
!mv "/content/test_data/test_img" "/content"
!mv "/content/test_img" "/content/test"

# remove unnecessary files
!rm -rf "train_data" "test_data" "sample_data"

**Set device (GPU or CPU)**

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Data Augmentation**

In [23]:
train_transforms = transforms.Compose(
    [
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomEqualize(p=0.5),
    transforms.RandomAutocontrast(p=0.5),
    transforms.RandomRotation(degrees=(-20,20)),
    transforms.CenterCrop(255),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3826, 0.3692, 0.3057], std=[0.3249, 0.3166, 0.3029])
    ]
)

test_transforms = transforms.Compose(
    [
    transforms.CenterCrop(255),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3820, 0.3704, 0.3064], std=[0.3255, 0.3179, 0.3040])
    ]
)

**Set Batch Size**

In [24]:
# set batch size
batch_size = 128

**Load Datasets**

In [25]:
# set directories for data
TRAIN_DATA_PATH = "/content/train"
VAL_DATA_PATH = "/content/val"
TEST_DATA_PATH = "/content/test"

# load data for each dataset
train_data = datasets.ImageFolder(root = TRAIN_DATA_PATH, transform=train_transforms)
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_data = datasets.ImageFolder(root = VAL_DATA_PATH, transform=test_transforms)
val_data_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

test_data = datasets.ImageFolder(root = TEST_DATA_PATH, transform=test_transforms)
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

**AlexNet 2.0**

In [26]:
class AlexNetV2(nn.Module):
    def __init__(self, in_channels = 3, hidden_size1 = 2048, num_classes = 10):
        super(AlexNetV2, self).__init__()
        # CONV layers
        self.conv1 = nn.Conv2d(3, 64, (3,3), stride=(1,1), padding=(1,1), bias=False)
        self.conv2 = nn.Conv2d(64, 128, (3,3), stride=(1,1), padding=(1,1), bias=False)
        self.conv3 = nn.Conv2d(128, 256, (3,3), stride=(1,1), padding=(1,1), bias=False)
        self.conv4 = nn.Conv2d(256, 512, (3,3), stride=(1,1), padding=(1,1), bias=False)
        self.conv5 = nn.Conv2d(512, 256, (3,3), stride=(1,1), padding=(1,1), bias=False)
        
        # MAXPOOL layers
        self.pool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2))
        
        # Batch Norm layers
        self.conv1_bn2 = nn.BatchNorm2d(64)
        self.conv2_bn2 = nn.BatchNorm2d(128)
        self.conv3_bn2 = nn.BatchNorm2d(256)
        self.conv4_bn2 = nn.BatchNorm2d(512)
        self.conv5_bn2 = nn.BatchNorm2d(256)
        self.fc1_bn1 = nn.BatchNorm1d(hidden_size1)

        # FC layers
        self.fc1 = nn.Linear(256*15*15, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes)

        # Dropout layers
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1_bn2(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_bn2(self.conv2(x))))
        x = self.pool(F.relu(self.conv3_bn2(self.conv3(x))))
        x = self.pool(F.relu(self.conv4_bn2(self.conv4(x))))
        x = F.relu(self.conv5_bn2(self.conv5(x)))
                
        x = x.reshape(x.shape[0], -1)

        x = self.dropout(x)
        x = F.relu(self.fc1_bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight)

                if m.bias is not None:
                    nn.init.constant_(m_bias, 0)

**Hyperparameters ,Optimizer, and Loss Function**

In [41]:
learning_rate = 0.001 
num_epochs = 20

model =  AlexNetV2().to(device)

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
loss_func = nn.CrossEntropyLoss()

**Data Structures to Save Data**

In [42]:
# epoch
epoch_num = [x for x in range(1,num_epochs+1)]

# training accuracy and loss
train_accuracy, total_train_loss = [], []

# validation accuracy and loss
val_accuracy, total_val_loss = [], []

**Model Training (and Val Loop)**

In [ ]:
for epoch in range(1, num_epochs+1):
    train_num_correct, train_num_samples = 0, 0
    val_num_correct, val_num_samples = 0, 0

    # training loop
    model.train()
    torch.backends.cudnn.benchmark = True
    tmp_train_loss = []
    for imgs, classes in train_data_loader:
        imgs, classes = imgs.to(device=device), classes.to(device=device)

        # Forward propagation
        output = model(imgs)
        t_loss = loss_func(output, classes)
        tmp_train_loss.append(t_loss.item())

        # Clear gradients
        optimizer.zero_grad()

        # Backprop (Calculating gradients)
        t_loss.backward()

        # Update parameters
        optimizer.step()

        with torch.no_grad():
            scores = model(imgs)
            _, predictions = scores.max(1)

            train_num_correct += (predictions==classes).sum()
            train_num_samples += predictions.size(0)

    train_acc = (train_num_correct)/(train_num_samples)*100
    train_accuracy.append(train_acc.cpu())
    tmp_train_loss = np.array(tmp_train_loss)
    epoch_train_loss = np.mean(tmp_train_loss)
    total_train_loss.append(epoch_train_loss)

    # validation loop
    model.eval()
    tmp_val_loss = []
    for imgs, classes in val_data_loader:
        imgs, classes = imgs.to(device=device), classes.to(device=device)

        # Forward propagation
        output = model(imgs)
        v_loss = loss_func(output, classes)
        tmp_val_loss.append(v_loss.item())

        # Clear gradients
        optimizer.zero_grad()
        
        # Backprop
        v_loss.backward()

        optimizer.step()

        with torch.no_grad():
            scores = model(imgs)
            _, predictions = scores.max(1)

            val_num_correct += (predictions==classes).sum()
            val_num_samples += predictions.size(0)
        
    val_acc = (val_num_correct)/(val_num_samples)*100
    val_accuracy.append(val_acc.cpu())
    tmp_val_loss = np.array(tmp_val_loss)
    epoch_val_loss = np.mean(tmp_val_loss)
    total_val_loss.append(epoch_val_loss)

    print(f'Epoch {epoch} | Train % = {train_acc} | Val % = {val_acc} | Train loss = {epoch_train_loss} | Val loss = {epoch_val_loss}')

**Test Set**

In [ ]:
test_num_correct = 0
test_num_samples = 0

model.eval()
with torch.no_grad():
    for img, classes in test_data_loader:
        img, classes = img.to(device=device), classes.to(device=device)

        output = model(img)
        _, predictions = output.max(1)
        test_num_correct += (predictions == classes).sum()
        test_num_samples += predictions.size(0)
    
    print(f'Test Accuracy ----------------------------> {(test_num_correct)/(test_num_samples)*100:.2f}%')

**Accuracy Chart (Train and Val)**

In [ ]:
plt.plot(epoch_num, train_accuracy, 'r', label="train")
plt.plot(epoch_num, val_accuracy, 'b', label="val")
plt.title('Accuracy for AlexNet v2.0 for Animals10')
plt.ylabel('Accuracy (%)')
plt.xlabel('Epochs')
plt.legend(loc="lower right")
#plt.rcParams['figure.figsize'] = [10, 5]
plt.show()

**Loss (Train and Val)**

In [ ]:
plt.plot(epoch_num, total_train_loss, 'r', label="train")
plt.plot(epoch_num, total_val_loss, 'b', label="val")
plt.title('Loss for AlexNet v2.0 for Animals10')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(loc="lower right")
plt.rcParams['figure.figsize'] = [10, 5]
plt.show()

**Summary of the model**

In [ ]:
summary(model, (3, 255, 255))